In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
import sklearn
import random
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_curve, auc, f1_score, accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.model_selection import RandomizedSearchCV
import ast
import os
from scipy import stats
import json
import joblib

warnings.filterwarnings("ignore")

#plt.style.use('ggplot')

In [ ]:
def train_val_split(deriv_data, shuffle=True, random_state=42,train_perc=0.85):
    # Divide patients to train / validation / groups
    
    #random.seed(random_state)
    # Divide patients to train / validation / groups
    
    patient_list = deriv_data['patient_id'].unique()
    
    if shuffle == True:
        random.shuffle(patient_list)
    
    # Calculate the number of items in each sublist
    total_items = len(patient_list)
    train_size = int(total_items * train_perc)
    val_size = total_items - train_size  # To ensure all items are included

    # Divide the list into sublists
    train_list = patient_list[:train_size]
    val_list = patient_list[train_size:]
    
    train_data = deriv_data[deriv_data['patient_id'].isin(train_list)].reset_index(drop=True)
    val_data = deriv_data[deriv_data['patient_id'].isin(val_list)].reset_index(drop=True)

    return train_data, val_data

In [ ]:
healthy_color = '#777777'
AML_color = '#BF9F45'       #'#E24A33'
MDS_color = '#348ABD'
MF_color = '#2b6e2a'       #'#155236'
any_MN_color = '#2d0e3d'

In [ ]:
AML_cmap = LinearSegmentedColormap.from_list('MF_cmap', ['#FFFFFF', AML_color])
MDS_cmap = LinearSegmentedColormap.from_list('MF_cmap', ['#FFFFFF', MDS_color])
MF_cmap = LinearSegmentedColormap.from_list('MF_cmap', ['#FFFFFF', MF_color])
any_MN_cmap = LinearSegmentedColormap.from_list('any_MN_cmap', ['#FFFFFF', any_MN_color])

In [ ]:
fs=13

In [ ]:
def reduce_train_data(train_data, shuffle=True, random_state=42, ratio=100):
    
    ## Reduce number of healthy datapoints -- 100 healthy controls per patient
    
    train_disease = train_data[train_data['disease_status'] == 1]
    train_healthy = train_data[train_data['disease_status'] == 0]
    n_train_d = len(train_disease['patient_id'].unique())
    n_train_h = n_train_d * ratio
    healthy_list = train_healthy['patient_id'].unique()
    
    #random.seed(random_state)
    
    if shuffle == True:
        random.shuffle(healthy_list)
    
    healthy_subset = healthy_list[:n_train_h]
    train_healthy_subset = train_healthy[train_healthy['patient_id'].isin(healthy_subset)].reset_index(drop=True)
    train_data = pd.concat([train_disease, train_healthy_subset], axis=0)

    return train_data

In [ ]:
my_path = '~/mounts/research/husdatalake/disease/scripts/Preleukemia/oona_new'

In [ ]:
disease = 'MF'

In [ ]:
if disease == 'de_novo_AML':
    cmap = AML_cmap
if disease == 'MDS':
    cmap = MDS_cmap
if disease == 'MF':
    cmap = MF_cmap
if disease == 'any_MN':
    cmap = any_MN_cmap

In [ ]:
method = 'youden'

In [ ]:
# Predicting if patient will get disease during the next X days
prediction_horizon = 365 *5

In [ ]:
# How many previous datapoints are needed for applying trajectory model
min_points=3

In [ ]:
# Whether to include hard positives
include_hp = True

In [ ]:
## Read first stage cox model binary threshold
import json

with open('results/basic_model/' + disease + '_threshold_youden.json', 'r') as f:
    thresholds = json.load(f)

binary_threshold = thresholds['med']

print(binary_threshold)

# 1. Read risk score feature data 

In [ ]:
if include_hp == True:
    features_df = pd.read_csv('trajectory_model/' + disease + '_full_risk_score_deriv_data_with_hp.csv', engine='c', low_memory=False)
else:
    features_df = pd.read_csv('trajectory_model/' + disease + '_full_risk_score_deriv_data.csv', engine='c', low_memory=False)

In [ ]:
features_df['score_gt_thresh'] = features_df['risk_score_now'] >= binary_threshold

In [ ]:
# Create label based on prediction horizon
features_df['label'] = np.where(
    (features_df['disease_status'] == 1) & (features_df['time_to_dg'] >= -prediction_horizon),
    1,
    0
)

In [ ]:
## Disease patients to positive
features_df.loc[features_df['label'] == 1, 'time_to_dg'] = features_df.loc[features_df['label'] == 1, 'time_to_dg'].abs()

In [ ]:
## Use only rows with n_prev > min_points
features_df = features_df[features_df['n_prev']>= min_points]

# Optimize hyperparams

In [ ]:
# Define the parameter grid
param_grid = {
    'objective' : ['survival:cox'],
    'eval_metric' : ['cox-nloglik'],
    'max_depth': [3, 5, 6],
    'learning_rate': [0.01, 0.05],
    'subsample': [0.6, 1.0],
    'colsample_bytree': [0.6, 1.0],
    'reg_alpha': [0, 0.5],
    'reg_lambda': [0.5, 2, 5],
    'tree_method': ['hist']
}

In [ ]:
ratio=100

In [ ]:

nrounds = 1000
early_stop = 20

In [ ]:
parameters_grid = ParameterGrid(param_grid)

In [ ]:
len(parameters_grid)

In [ ]:
result_df = pd.DataFrame(columns=['params', 'AUC_mean', 'AUC_std', 'AUCPR_mean', 'AUCPR_std'])

In [ ]:
cv=10

In [ ]:

for params in parameters_grid:
    
    print(f'\nTRAINING MODEL WITH PARAMETERS:\n {params} \n{cv}-FOLD CROSS VALIDATION')
    
    AUCs = []
    AUCPRs = []
    
    for i in range(cv):
    
        rs = random.randint(1, 1000)
        print(rs)
        
        train, val = train_val_split(features_df,random_state=rs, train_perc=0.85)
        
        # Sanity check - is any of test indices in validation or training sets
        print('\nSanity check: Is there any validaion data in train set')
        train_ht = list(train['patient_id'].unique())
        validation_ht = list(val['patient_id'].unique())
        #test_ht = list(test_data['henkilotunnus'].unique())
        val_in_train = np.intersect1d(validation_ht, train_ht).size > 0
        print(val_in_train)
        
        # Train classifier on rows with enough prior data
        train = train[train['n_prev'] >= min_points].copy()
        
        # <ratio> controls per 1 patient
        print('N train data rows before reduction: ', len(train))
        train = reduce_train_data(train, ratio=ratio)
        print('N train data rows after reduction: ', len(train))
        
        X_train = train.drop(columns=['patient_id', 'disease_status', 'time_to_dg', 'label', 'score_gt_thresh'])
        y_train = train['time_to_dg']
        X_val = val.drop(columns=['patient_id', 'disease_status', 'time_to_dg', 'label', 'score_gt_thresh'])
        y_val = val['time_to_dg']
        
        # Create DMatrix for XGBoost
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        
        # Use validation set to watch performance
        watchlist = [(dtrain,'train'), (dval,'eval')]
        
        # Store validation results
        evals_results = {}
        
        # Train the model
        xgb_model = xgb.train(params, dtrain, num_boost_round=nrounds, early_stopping_rounds=early_stop, evals=watchlist, evals_result=evals_results, verbose_eval=50)
        
        # Predict risk scores
        risk_scores_train = xgb_model.predict(dtrain)
        risk_scores_val = xgb_model.predict(dval)
        
        # Add risk scores to the dataframe
        train['risk_score'] = risk_scores_train
        val['risk_score'] = risk_scores_val
    
        # Predict risk scores
        risk_scores_train = xgb_model.predict(dtrain)
        risk_scores_val = xgb_model.predict(dval)
        
        # Add risk scores to the dataframe
        train['risk_score'] = risk_scores_train
        val['risk_score'] = risk_scores_val
        
        # Calculate C-index for validation set
        # Negative times to positive for getting c-index
        val['time_to_dg'] = val['time_to_dg'].apply(lambda x: -x if x < 0 else x)
        
        try:
            # ROC-AUC
            fpr, tpr, thresholds = roc_curve(val['label'], val['risk_score'])
            roc_auc = auc(fpr, tpr)
            AUCs.append(roc_auc)

            # AUCPR
            precision, recall, _ = precision_recall_curve(val['label'], val['risk_score'])
            aucpr = auc(recall, precision)
            AUCPRs.append(aucpr)


        except:
            print('Something went wrong with model training with current parameters.')
    
    result_df.loc[len(result_df.index)] = [params, np.mean(AUCs), np.std(AUCs), np.mean(AUCPRs), np.std(AUCPRs)]
    # Save results at each iteration
    result_df.to_csv('trajectory_model/' + disease + '_hyperparameter_results_cv.csv')